# Production Serving Benchmark - Bridge Failure Prediction

In [1]:
import requests
import time
import pandas as pd
import numpy as np

## Prepare Batch Payload

In [2]:
df = pd.read_csv('../data/processed/features.csv')
payload = df.drop(['failure_within_1yr'], axis=1).to_dict(orient='records')

## Benchmark API Throughput and Latency

In [3]:
url = 'http://localhost:8080/predict'
headers = {'Authorization': 'Bearer DEMO_TOKEN'}

batch_sizes = [1, 10, 100, 500]
results = []

for batch in batch_sizes:
    samples = payload[:batch]
    start = time.time()
    resp = requests.post(url, json=samples, headers=headers)
    elapsed = time.time() - start
    results.append({'batch_size': batch, 'elapsed_sec': elapsed, 'status': resp.status_code})
    print(f'Batch {batch}: {elapsed:.3f}s, status {resp.status_code}')

## Plot Latency vs Batch Size

In [4]:
import matplotlib.pyplot as plt
df_bench = pd.DataFrame(results)
plt.plot(df_bench['batch_size'], df_bench['elapsed_sec'], marker='o')
plt.xscale('log')
plt.xlabel('Batch Size (log scale)')
plt.ylabel('Elapsed Time (s)')
plt.title('API Latency vs Batch Size')
plt.show()